In [ ]:
!pip install pytorch-pretrained-bert==0.4.0
!pip install pytorch_transformers
!pip install transformers
!pip install botocore

In [ ]:
!pip install bcolz
!pip install torchnlp

In [ ]:
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
import theano
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from pytorch_pretrained_bert import BertTokenizer, BertModel
from pytorch_transformers.modeling_bert import BertPreTrainedModel
from transformers import BertConfig, BertTokenizer,TFBertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from torch import nn
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import csv
from operator import itemgetter
import gzip
import bcolz
import pickle
from collections import Counter
import string
import warnings
from pandas.core.common import SettingWithCopyWarning
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext
from torch.utils.data import DataLoader
from torch.utils import data
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
%matplotlib inline

### Data (Train, Validation, Test)

In [ ]:
train_data = pd.read_csv('product_10k_train.csv')
val_data = pd.read_csv('product_10k_val.csv')
test_data = pd.read_csv('product_10k_test.csv')

### Preprocess

In [ ]:
## --------------------------------- TWO INPUTS --------------------------------------------- ##
## -------------------------- -1. TEXT COLUMN. 2. MAX_LEN ----------------------------------- ##
def preprocess_for_bert(data_text, MAX_LEN = 120): 
    tokenizer=BertTokenizer.from_pretrained('bert-base-uncased-vocab.txt',do_lower_case=True, return_tensors='np')
    tokenized_sents=[tokenizer.tokenize(sent) for sent in ['[CLS] ' + sent + ' [SEP]' for sent in data_text]]
    input_ids=pad_sequences([tokenizer.convert_tokens_to_ids(sent) for sent in tokenized_sents], 
                            maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []
    for seq in input_ids:
        attention_masks.append([int(i>0) for i in seq])
    # Convert lists to tensors
    input_ids = torch.LongTensor(input_ids)
    attention_masks = torch.LongTensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Current device is {dev}")

Current device is cuda


In [ ]:
X_train, y_train = np.array(train_data['reviewText'].astype("str")),  np.array(train_data['helpfulness']).astype('float')
X_val,  y_val = np.array(val_data['reviewText'].astype("str")),  np.array(val_data['helpfulness']).astype('float')
X_test, y_test = np.array(test_data['reviewText'].astype("str")),  np.array(test_data['helpfulness']).astype('float')
                                 
train_inputs, train_masks = preprocess_for_bert(X_train)
val_inputs, val_masks = preprocess_for_bert(X_val)
test_inputs, test_masks = preprocess_for_bert(X_test)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train).float()
val_labels = torch.tensor(y_val).float()
test_labels = torch.tensor(y_test).float()


# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [ ]:
from transformers import  BertConfig
config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)
model = BertModel.from_pretrained("bert-base-uncased")

In [ ]:
# Create the BertRegressor class
class BertRegressor(nn.Module):
    """Bert Model for Regression Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module regressor
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertRegressor, self).__init__()
        # Specify hidden size of BERT, hidden size of our regressor, and number of labels
        D_in, H, D_out = 768, 50, 1

        # Instantiate BERT model
        self.bert = model

        # Instantiate an one-layer feed-forward regressor
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the regressor to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        choosed_hidden_state_cls = outputs[0][10][0]
        # Feed input to classifier to compute logits
        logits = self.classifier(choosed_hidden_state_cls)

        return logits

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Regressor, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Regressor
    bert_regressor = BertRegressor(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_regressor.to(dev)

    # Create the optimizer
    optimizer = AdamW(bert_regressor.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_regressor, optimizer, scheduler

In [ ]:
from torch.nn import MSELoss
import random
import time

# Specify loss function
loss_fn = nn.MSELoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, test_dataloader = None, epochs=4, evaluation=False, test_eva = True):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Test Loss':^10} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(dev) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            logits=logits.double()
            b_labels=b_labels.double()
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {'-':^9}| {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss = evaluate(model, val_dataloader)

        if test_eva == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            test_loss = test_evaluate(model, test_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {test_loss:^10.6f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")

        
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
#     val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(dev) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
#         accuracy = (preds == b_labels).cpu().numpy().mean() * 100
#         val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
#     val_accuracy = np.mean(val_accuracy)

    return val_loss

def test_evaluate(model, test_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
#     val_accuracy = []
    test_loss = []

    # For each batch in our validation set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(dev) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        test_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
#         accuracy = (preds == b_labels).cpu().numpy().mean() * 100
#         val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    test_loss = np.mean(test_loss)
#     val_accuracy = np.mean(val_accuracy)

    return test_loss

In [ ]:
set_seed(2018)    # Set seed for reproducibility
bert_regressor, optimizer, scheduler = initialize_model(epochs=2)
train(bert_regressor, train_dataloader, val_dataloader, test_dataloader, epochs=2, evaluation=True, test_eva = True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  | Test Loss  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.054089   |     -      |     -     |     -    |   8.55   
   1    |   40    |   0.051440   |     -      |     -     |     -    |   8.11   
   1    |   60    |   0.055667   |     -      |     -     |     -    |   8.14   
   1    |   80    |   0.047903   |     -      |     -     |     -    |   8.12   
   1    |   100   |   0.052916   |     -      |     -     |     -    |   8.12   
   1    |   120   |   0.052617   |     -      |     -     |     -    |   8.11   
   1    |   140   |   0.054489   |     -      |     -     |     -    |   8.12   
   1    |   160   |   0.050928   |     -      |     -     |     -    |   8.11   
   1    |   180   |   0.049474   |     -      |     -     |     -    |   8.11   
   1    |   200   |   0.048536   |     -      |     -     |     -    |   8.12   
   1    |   220   |   0.04959

## Classification Task